In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Flatten, Conv2D, MaxPooling2D
import warnings
warnings.filterwarnings('ignore')

In [160]:
df = pd.read_csv('https://raw.githubusercontent.com/ghassenabdedayem/AltCoins/master/data_sources/Crypto/BTC_data.csv', sep = ';')
df.head()

,BTC_Currency,BTC_Date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,BTC_Market_Cap
0,bitcoin,"Dec 04, 2019","7,320.13","7,539.78","7,170.92","7,252.03","21,664,240,918","131,143,073,943"
1,bitcoin,"Dec 03, 2019","7,323.98","7,418.86","7,229.36","7,320.15","14,797,485,769","132,359,942,301"
2,bitcoin,"Dec 02, 2019","7,424.04","7,474.82","7,233.40","7,321.99","17,082,040,706","132,378,982,377"
3,bitcoin,"Dec 01, 2019","7,571.62","7,571.62","7,291.34","7,424.29","18,720,708,479","134,215,145,410"
4,bitcoin,"Nov 30, 2019","7,764.06","7,836.10","7,515.85","7,569.63","17,158,194,786","136,829,391,049"


In [161]:
df = df.drop(labels=['BTC_Currency'], axis = 1)
df.head()

,BTC_Date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,BTC_Market_Cap
0,"Dec 04, 2019","7,320.13","7,539.78","7,170.92","7,252.03","21,664,240,918","131,143,073,943"
1,"Dec 03, 2019","7,323.98","7,418.86","7,229.36","7,320.15","14,797,485,769","132,359,942,301"
2,"Dec 02, 2019","7,424.04","7,474.82","7,233.40","7,321.99","17,082,040,706","132,378,982,377"
3,"Dec 01, 2019","7,571.62","7,571.62","7,291.34","7,424.29","18,720,708,479","134,215,145,410"
4,"Nov 30, 2019","7,764.06","7,836.10","7,515.85","7,569.63","17,158,194,786","136,829,391,049"


In [18]:
df.BTC_Date = pd.to_datetime(df.BTC_Date)
df = df.set_index('BTC_Date')
df.head()

,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,BTC_Market_Cap
BTC_Date,,,,,,
2019-12-04,"7,320.13","7,539.78","7,170.92","7,252.03","21,664,240,918","131,143,073,943"
2019-12-03,"7,323.98","7,418.86","7,229.36","7,320.15","14,797,485,769","132,359,942,301"
2019-12-02,"7,424.04","7,474.82","7,233.40","7,321.99","17,082,040,706","132,378,982,377"
2019-12-01,"7,571.62","7,571.62","7,291.34","7,424.29","18,720,708,479","134,215,145,410"
2019-11-30,"7,764.06","7,836.10","7,515.85","7,569.63","17,158,194,786","136,829,391,049"


In [21]:
colonnes = ['BTC_Open', 'BTC_High', 'BTC_Low', 'BTC_Close', 'BTC_Volume', 'BTC_Market_Cap']
for j in colonnes:
  for i in range(len(df)):
    if type(df[j][i]) == str:
      df[j][i]=df[j][i].replace(',','')
      df[j][i]=pd.to_numeric(df[j][i])
      df[j][i] = np.float(df[j][i]) #convert numerical to integer in order to be able to apply statsmodels timeseries seasonal decompose
df.head()

,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,BTC_Market_Cap
BTC_Date,,,,,,
2019-12-04,7320.13,7539.78,7170.92,7252.03,2.16642e+10,1.31143e+11
2019-12-03,7323.98,7418.86,7229.36,7320.15,1.47975e+10,1.3236e+11
2019-12-02,7424.04,7474.82,7233.4,7321.99,1.7082e+10,1.32379e+11
2019-12-01,7571.62,7571.62,7291.34,7424.29,1.87207e+10,1.34215e+11
2019-11-30,7764.06,7836.1,7515.85,7569.63,1.71582e+10,1.36829e+11


In [0]:
n_input = 40
n_features = 6
train, test = df[n_input:], df[:n_input]

In [0]:
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

In [0]:
generator = TimeseriesGenerator(train, train[:,1], length=n_input, batch_size=6)

In [151]:
train[:,1].shape

(2372,)

In [134]:
# train = np.ndarray(train)
type(train)

pandas.core.frame.DataFrame

In [156]:
train.head()

AttributeError: ignored

In [0]:
generator = TimeseriesGenerator(train, train['BTC_High'], length=n_input, batch_size=6)

In [26]:
#####################################
generator_multi = TimeseriesGenerator(train, train[:], length=n_input, batch_size=6)
print(generator_multi)
print(generator)

In [148]:
train['BTC_High'].shape

(2372,)

Pour tester, je reprends le même modèle SLTM. Il faudra voir aussi dans quelle mesure il est possible d'utiliser des modèles autres que les Réseaux de Neurones. Je n'ai pas encore testé si le "generator" le permet.

In [153]:
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit_generator(generator, epochs=3)

Epoch 1/3
389/389 [==============================] - 23s 58ms/step - loss: 0.0024
Epoch 2/3
389/389 [==============================] - 23s 59ms/step - loss: 0.0011
Epoch 3/3
389/389 [==============================] - 22s 57ms/step - loss: 7.9317e-04


In [0]:
# i = 0
# for elt in generator_multi:
#   print(elt[1][0][1])


In [0]:
##########################################""
model_multi = Sequential()

model_multi.add(Conv2D(filters=30,
                 kernel_size=(5,5), 
                 border_mode='valid', 
                 activation='relu', 
                 data_format='channels_first'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit_generator(generator, epochs=1)

In [0]:
print(type(train))

<class 'pandas.core.frame.DataFrame'>


In [0]:
train.head()

,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,BTC_Market Cap
BTC_Date,,,,,,
2019-10-25,7490.7,8691.54,7479.98,8660.7,2.87051e+10,1.56007e+11
2019-10-24,7509.73,7532.87,7446.99,7493.49,1.62687e+10,1.34968e+11
2019-10-23,8076.23,8093,7469.32,7514.67,2.19429e+10,1.35337e+11
2019-10-22,8243.4,8296.65,8074.46,8078.2,1.68034e+10,1.4547e+11
2019-10-21,8225.12,8296.69,8196.42,8243.72,1.58687e+10,1.48433e+11


In [32]:
train_array = np.array(train)
print(train_array[:n_input,:].shape)

(40, 6)


In [0]:
pred_list = []

batch = train_array[:n_input,:].reshape((1, n_input, n_features))

for i in range(n_input):
  pred_list.append(model.predict(batch)[0])
  batch = np.append(batch[:, :-1,:], [[pred_list[i]]], axis=1)

  # batch = np.append(batch[:, :-1,:], [[pred_list[i][0]]], axis=1)

In [47]:
print((pred_list))

[array([0.5099181], dtype=float32)]


In [0]:
list_test = [test[39][0], pred_list[0][0], test[39][2], test[39][3], test[39][4], test[39][5]]
# list_test.append(pred_list[0][0])

In [105]:
print(list_test)

[0.4430848185991869, 0.5099181, 0.4546663232597706, 0.4723122224183783, 0.9864878027442234, 0.508924462931676]


In [103]:
print(test[39][0])

0.4430848185991869


In [106]:
print(np.array(list_test).reshape((1,6)).shape)
test_reshaped = np.array(list_test).reshape((1,6))

(1, 6)


In [0]:
test_unscaled = scaler.inverse_transform(test_reshaped)

In [108]:
print(test[0].reshape((1,6)).shape)

(1, 6)


In [109]:
print(scaler.inverse_transform(test[0].reshape((1,6))))
print(test_unscaled)

[[7.32013000e+03 7.53978000e+03 7.17092000e+03 7.25203000e+03
  2.16642409e+10 1.31143074e+11]]
[[8.66758000e+03 1.02802851e+04 8.66262000e+03 9.24497000e+03
  4.44962556e+10 1.66547361e+11]]


In [48]:
pred_list_unscaled = scaler.inverse_transform(pred_list)

ValueError: ignored

In [50]:
print(batch[:, :-1,:][0])

[[0.38244372 0.43053815 0.39212114 0.44224012 0.63639505 0.47656346]
 [0.38342428 0.37264645 0.39037643 0.38216437 0.3606794  0.41197446]
 [0.41261433 0.40063274 0.39155737 0.38325449 0.48647649 0.41310718]
 [0.42122809 0.4108079  0.42356085 0.41225912 0.37253308 0.44421417]
 [0.42028618 0.4108099  0.43001084 0.42077835 0.35181224 0.45331105]
 [0.40857358 0.41006693 0.41694903 0.41966455 0.34373124 0.45206418]
 [0.40733641 0.40011462 0.41670259 0.40764539 0.3058996  0.43911324]
 [0.41388704 0.40290161 0.41444858 0.40685533 0.34699785 0.43822128]
 [0.41114993 0.40272273 0.41967267 0.4135824  0.31732224 0.44539411]
 [0.41923245 0.40681878 0.41883442 0.41068055 0.35631051 0.44223192]
 [0.4279297  0.41650678 0.42928577 0.4188045  0.33743854 0.45090825]
 [0.42521783 0.41548252 0.43464947 0.42751932 0.33590825 0.46021188]
 [0.42604587 0.41951861 0.43425177 0.42475643 0.30613151 0.45720418]
 [0.42495144 0.41673312 0.43619428 0.42555678 0.32219056 0.45801759]
 [0.43884312 0.43204906 0.43634447

In [45]:
print(batch[:, :-1,:][0][0])

[0.38244372 0.43053815 0.39212114 0.44224012 0.63639505 0.47656346]


In [0]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), 
                          index=df[:n_input].index,
                          columns = ['Predict'])

In [0]:
df_test = pd.concat([df, df_predict], axis = 1)
df_test.tail(20)

In [0]:
plt.figure(figsize=(15, 10))
plt.plot(df_test.index, df_test['High'])
plt.plot(df_test.index, df_test['Predict'], color='r')
plt.show()